# Homework 8

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
from gd import rewrite_as_quadratic_form

# Inline plotting
%matplotlib inline

# Make sympy print pretty math expressions
sy.init_printing()

---
## Exercise 8.8


<img src="figures/homework-08/exercise-8.8.png" width="600" />


In [2]:
x1, x2 = sy.symbols('x1, x2')
x = sy.Matrix([[x1],
               [x2]])
f = 3*(x1**2  + x2**2)  +   4*x1*x2  + 5*x1  + 6*x2 + 7
f

First, we need to rewrite the function $f$ as the following quadratic form:
$$
f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^T \mathbf{Q} \mathbf{x} - \mathbf{b}^T \mathbf{x}
$$

In [3]:
Q, b = rewrite_as_quadratic_form(f, x1, x2)

Ignoring constant term: 7


In [4]:
Q

⎡6  4⎤
⎢    ⎥
⎣4  6⎦

Let us ensure that the return $\mathbf{Q}$ and $\mathbf{b}$ are correct:

In [5]:
sy.simplify(1/2 * x.T * Q * x - b.T * x)

⎡      2                              2         ⎤
⎣3.0⋅x₁  + 4.0⋅x₁⋅x₂ + 5.0⋅x₁ + 3.0⋅x₂  + 6.0⋅x₂⎦

The only difference with this result and the original function $f$ is the constant term. 

In [6]:
f

Next, we find the characteristic polynomial of $\mathbf{Q}$ which is given by $\det(\lambda I - Q)$:

In [7]:
lamda = sy.symbols('lambda')
char_poly = sy.det(lamda * sy.eye(2) - Q)
char_poly

Solving the characteristic polynomial, we get:

In [8]:
sy.nonlinsolve([char_poly], lamda)

From this, we know that $\lambda = 2$ and $\lambda = 10$. This means that eigenvalues of $\mathbf{Q}$ are $2$ and $10$. 

We can use SymPy to find the characteristic polynomial and eigenvalues directly:

In [9]:
Q.charpoly()

In [10]:
Q.eigenvals()

The result is interpreted as follows; the matrix $\mathbf{Q}$ has the eigenvalues 2 and 10 and that the eigenvalues 2 and 10 have algebraic multiplicity 1. Algebraic multiplicity is the number of times that an eigenvalue appears in the characteristic polynomial of a matrix. This is important because there are cases where the same eigenvalue appears multiple tiles in the characteristic polynomial of a matrix. 

In [11]:
sy.Matrix([[5, 0],
           [0, 5]]).eigenvals()

To find the range of values for $\alpha$, we use the Theorem 8.3:


<img src="figures/lecture-08/theorem-8.3.png" width="600" />


Since $\lambda_{max}(\mathbf{Q}) = 10$, the largest range of values of $\alpha$ for which the algorithm is globally convergent is $0 < \alpha < 2/10$

---
## Exercise 8.16


<img src="figures/homework-08/exercise-8.16.png" width="600" />


---
<img src="figures/homework-08/exercise-8.16-a.png" width="600" />


We know that $\lVert \mathbf{x} \rVert^2 = \mathbf{x}^T \mathbf{x}$ where $\mathbf{x} \in \mathbb{R}^n$ because by definition  $\lVert \mathbf{x} \rVert = \sqrt{\mathbf{x}^T \mathbf{x}}$, we have:

\begin{align}
\lVert \mathbf{Ax-b} \rVert^2 
&= (\mathbf{Ax} - \mathbf{b})^T(\mathbf{Ax} - \mathbf{b}) \\
&= (\mathbf{Ax})^T (\mathbf{Ax}) - (\mathbf{Ax})^T \mathbf{b} - \mathbf{b}^T(\mathbf{Ax})  + \mathbf{b}^T \mathbf{b} \\
&= \mathbf{x}^T \mathbf{A}^T (\mathbf{Ax}) - 2 (\mathbf{Ax})^T \mathbf{b}  + \mathbf{b}^T \mathbf{b} \\
&= \mathbf{x}^T (\mathbf{A}^T \mathbf{A})\mathbf{x} - 2 \mathbf{b}^T(\mathbf{Ax})  + \mathbf{b}^T \mathbf{b}
\end{align}

The gradient of the function is:
\begin{align}
f'(\mathbf{x}) = 2 (\mathbf{A}^T \mathbf{A})\mathbf{x} - 2 \mathbf{b}^T \mathbf{A}
\end{align}

Notice that $\mathbf{Q} = 2(\mathbf{A}^T \mathbf{A})$ because


<img src="figures/lecture-08/solution-steepest-descent-for-quadratic-functions.png" width="600" />


The Hessian of the function is:
\begin{align}
f''(\mathbf{x}) = 2 (\mathbf{A}^T \mathbf{A})
\end{align}

Let us verify our result using SymPy:

In [12]:
m, n = sy.symbols('m, n')
A = sy.MatrixSymbol('A', m, n)
b = sy.MatrixSymbol('b', m, 1)
x = sy.MatrixSymbol('x', n, 1)

# Turn it into a Matrix
f = sy.Matrix([ (A*x - b).T * (A*x-b) ])
f

⎡⎛  T    T  T⎞           ⎤
⎣⎝-b  + x ⋅A ⎠⋅(-b + A⋅x)⎦

In [13]:
# Find the gradient of the function
gradient = f.jacobian([x])
gradient

⎡   T           ⎤
⎣2⋅A ⋅(-b + A⋅x)⎦

In [14]:
# Find the Hessian of the function
hessian = gradient.jacobian([x])
hessian

⎡   T  ⎤
⎣2⋅A ⋅A⎦

---
<img src="figures/homework-08/exercise-8.16-b.png" width="600" />


In general, the fixed-step-size algorithm is given by:


<img src="figures/lecture-08/fixed-step-size-gradient-algorithm.png" width="600" />


where $\mathbf{g}^{(k)}$ is gradient of the function i.e., $\nabla f(\mathbf{x}^{(k)})$

The fixed-step-size gradient algorithm for solving is therefore:
$$
\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} - \alpha (  2 (\mathbf{A}^T \mathbf{A})\mathbf{x} - 2 \mathbf{b}^T \mathbf{A}  )
$$

---
<img src="figures/homework-08/exercise-8.16-c.png" width="600" />


In [15]:
A = sy.Matrix([[1, 0],
               [0, 2]])
# Find Q
Q = 2* (A.T * A)

# Compute the eigenvalues of Q
Q.eigenvals()

We use Theorem 8.3:


<img src="figures/lecture-08/theorem-8.3.png" width="600" />


Thus, $0 < \alpha < \frac{2}{8}$

---
## Exercise 8.21


<img src="figures/homework-08/exercise-8.21.png" width="600" />


---
<img src="figures/homework-08/exercise-8.21-a.png" width="600" />


---
<img src="figures/homework-08/exercise-8.21-b.png" width="600" />


---
## Exercise 8.25

<img src="figures/homework-08/exercise-8.25.png" width="600" />


---
## Exercise 8.26

<img src="figures/homework-08/exercise-8.26.png" width="600" />
